# Network motif motivating examples

In [3]:
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [4]:
import sys
sys.path.append('/data/home/Github/scrna-parameter-estimation/dist/memento-0.0.4-py3.7.egg')
import memento

In [5]:
data_path = '/data_volume/ifn_hbec/'

### Read the processed RNA data

Focus on the club and bc/club cells and type I interferons for now.

Encode the timestamps to integers.

In [7]:
adata = sc.read(data_path + 'HBEC_type_I_filtered_counts.h5ad')

In [8]:
adata = adata[
    adata.obs.cell_type.isin(['club', 'basal/club']) & \
    adata.obs.stim.isin(['control','beta'])].copy()

### Get some 1D moments

In [9]:
adata.obs['q'] = 0.25*0.2
memento.setup_memento(adata, q_column='q')

### Run memento for each subset, comparing to control

In [9]:
cts = [['ciliated'], ['bc', 'club'], ['basal']]
tps = ['3', '6', '9', '24', '48']
stims = ['alpha', 'beta', 'gamma', 'lambda']

In [ ]:
for ct in cts:
    for tp in tps:
        for stim in stims:
            
            try:
            
                print('starting', ct, tp, stim)

                adata_stim = adata.copy()[
                    adata.obs.cell_type.isin(ct) & \
                    adata.obs.stim.isin(['control', stim]) & \
                    adata.obs.time.isin(['0',tp]), :].copy()
                time_converter={0:0, int(tp):1}
                adata_stim.obs['time_step'] = adata_stim.obs['time'].astype(int).apply(lambda x: time_converter[x])

                memento.create_groups(adata_stim, label_columns=['time_step', 'donor'])
                memento.compute_1d_moments(adata_stim, min_perc_group=.9)

                memento.ht_1d_moments(
                    adata_stim, 
                    formula_like='1 + time_step',
                    cov_column='time_step', 
                    num_boot=10000, 
                    verbose=1,
                    num_cpus=13)

                del adata_stim.uns['memento']['mv_regressor']

                adata_stim.write('/data_volume/ifn_hbec/binary_test/{}_{}_{}.h5ad'.format(
                    '-'.join(ct), stim, tp))
            except:
                continue